In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import math
from matplotlib.image import NonUniformImage
import cv2
from itertools import zip_longest
import numpy as np
import cv2
import math
import os 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
import time
from scipy.ndimage import median_filter


In [2]:
img_width, img_height = 128, 128

In [3]:
class MRELBP():
    def __init__(self, r=[2, 4, 6, 8], p = 8, w_c = 3, w_r = [3, 5, 7, 9]):
        self.r = r
        self.p = p
        self.w_c = w_c
        self.w_r = w_r
        # step :
        #   Median
        #   CI
        #   Bi-interpolation -> RI & ND
        #   Histogram

    def median_processing(self, image):
        m_3x3 = median_filter(image, size = self.w_r[0], mode='constant', cval=0)
        m_5x5 = median_filter(image, size = self.w_r[1], mode='constant', cval=0)
        m_7x7 = median_filter(image, size = self.w_r[2], mode='constant', cval=0)
        m_9x9 = median_filter(image, size = self.w_r[3], mode='constant', cval=0)
        return m_3x3, m_5x5, m_7x7, m_9x9



    def mrelbp_ci(self, image, in_r):
        # r = 2 -> window_size = 5

        width, height = image.shape
        out = np.zeros((width - 2 * in_r, height - 2 * in_r), dtype=np.uint8)

        x = int(in_r /  2)

        pixel_central =  np.zeros((width - 2 * in_r, height - 2 * in_r))
        muy = np.mean(image)

        for i in range(0,height - 2 * in_r):
            for j in range(0,width - 2 * in_r):

                pixel_central[i, j] = image[i + 2 * x][j + 2 * x]
                out[i, j] = 1 if pixel_central[i, j] >= muy else 0
        return out




    def getInterpolation(self, image, x, y, r):
        x1 = int(np.floor(x))
        x2 = int(np.ceil(x))
        y1 = int(np.floor(y))
        y2 = int(np.ceil(y))

        a = x - x1
        b = y - y1
        r1 = (1 - a) * (1-b)
        r2 =  a * (1 - b)
        r3 = (1 - a) * b
        r4 = a * b

        interpolated_value = (
            image[x1, y1] * r1 +
            image[x1, y2] * r2 +
            image[x2, y1] * r3 +
            image[x2, y2] * r4
        )
        return interpolated_value
    def jointHistogram(self, ci, ni, rd):
        width, height = ci.shape
        out_matrix = np.zeros(200, dtype=np.uint32)
        for i in  range(0, width):
            for j in range(0, height):
                out_matrix[ci[i, j] * 100 + ni[i, j] * 10 + rd[i, j]] += 1
        return out_matrix


    def getNIDescriptor(self, NI, r, sum):
        ni_des = 0
        for index in range(1, 9):
            scale_value = NI[index] * ((2 * r + 1) ** 2)
            if scale_value >= sum:
                ni_des  = ni_des + 2**(index - 1)
        return ni_des

    def getRDDescriptor(self, r2, r1):
        sum = 0
        for i in range(1, 9):
            if r2[i] >= r1[i]:
                sum = sum + 2**(i - 1)

        return sum
    def ror(self, pixel):
        min_value = 255
        binary_pixel = bin(pixel)[2:].zfill(8)
        binary_pixel_int = int(binary_pixel, 2)

        for i in range(8):
            result = ((binary_pixel_int >> i) | ((binary_pixel_int << (8 - i))) & 0xFF)
            if min_value > result:
                min_value = result
        return  min_value

    def checkU2(self, pixel):

        binary_pixel = bin(pixel)[2:].zfill(8)

        transitions = 0

        for i in range(8):
            current_bit = binary_pixel[i]
            next_bit = binary_pixel[(i + 1) % 8]  #

            if current_bit == '1' and next_bit == '0':
                transitions += 1
            elif current_bit == '0' and next_bit == '1':
                transitions += 1

        # Print the results
        return transitions
    def getSumPixel(self, pixel):
        sum = 0
        binary_pixel = bin(pixel)[2:].zfill(8)
        for i in range(0, 8):
            sum += int(binary_pixel[i])
        return sum


    def NI_RD_descriptor(self, image_r1, image_r2, r2):
        NI, RD = self.getNI_RD(image_r1, image_r2, r2)
        # print(NI)
        # print(RD)
        NI_out  = np.zeros(NI.shape, dtype= np.uint8)
        RD_out  = np.zeros(NI.shape, dtype= np.uint8)
        NI_width, NI_height = NI.shape
        for i in range(0, NI_height):
            for j in range(0, NI_width):
                NI_min = self.ror(np.uint8(NI[i, j]))
                NI_transitions = self.checkU2(NI_min)
                NI_des = 0
                if NI_transitions <= 2:
                    NI_des = self.getSumPixel(NI_min)
                else:
                    NI_des = 9
                NI_out[i, j] = NI_des

                RD_min  = self.ror(np.uint8(RD[i, j]))
                RD_transitions = self.checkU2(RD_min)
                RD_des = 0
                if RD_transitions <= 2:
                    RD_des = self.getSumPixel(RD_min)
                else:
                    RD_des = 9
                RD_out[i, j] = RD_des
        return NI_out, RD_out
    def MRELBP(self, image):
        start_time = time.time()
        m_3x3, m_5x5, m_7x7, m_9x9 = self.median_processing(image)

        # r = 2
        ci_r2 = self.mrelbp_ci(m_3x3, 2)
        ci_r4 = self.mrelbp_ci(m_3x3, 4)
        ci_r6 = self.mrelbp_ci(m_3x3, 6)
        ci_r8 = self.mrelbp_ci(m_3x3, 8)



        NI_r2, RD_r2 = self.NI_RD_descriptor(image, m_3x3, 2)
        NI_r4, RD_r4 = self.NI_RD_descriptor(m_3x3, m_5x5, 4)
        NI_r6, RD_r6 = self.NI_RD_descriptor(m_5x5, m_7x7, 6)
        NI_r8, RD_r8 = self.NI_RD_descriptor(m_7x7, m_9x9, 8)

        hist_r2 = self.jointHistogram(ci_r2, NI_r2, RD_r2)
        hist_r4 = self.jointHistogram(ci_r4, NI_r4, RD_r4)
        hist_r6 = self.jointHistogram(ci_r6, NI_r6, RD_r6)
        hist_r8 = self.jointHistogram(ci_r8, NI_r8, RD_r8)

        hist_o = np.concatenate((hist_r2, hist_r4, hist_r6, hist_r8))
        end_time = time.time()

        execution_time = end_time - start_time
        print(f"Execution Time: {execution_time:.4f} seconds")

        return hist_o



    def getInterNeighbors(self, image, r, i, j):
        if r == 1:
            S =  np.zeros(9, dtype = np.uint8)
            S[1] = image[i, j + r]
            S[2] = image[i -r , j + r]
            S[3] = image[i -r, j]
            S[4] = image[i -r, j - r]
            S[5] = image[i, j -r ]
            S[6] = image[i + r, j - r]
            S[7] = image[i + r , j]
            S[8] = image[i + r, j + r]
            return S
        angles = [45, 135, 225, 315]
        results = {}
        S =  np.zeros(9)
        S[1] = image[i, j + r]
                # S2 -> 45
        S[3] = image[i - r, j]
        S[5] = image[i, j - r]
        S[7] = image[i + r, j]
        # print(S[1], S[3], S[5], S[7])

        for angle in angles:
            theta = np.radians(angle)
            target_x = i - r * np.sin(theta)
            target_y = j + r * np.cos(theta)
                    # print(target_x, target_y)
            results[f"{angle}"] = self.getInterpolation(image, target_x, target_y, r)

        S[2] = results["45"]
        S[4] = results["135"]
        S[6] = results["225"]
        S[8] = results["315"]

        return S

    def getNI_RD(self, image_r1, image_r2, r2):
        width, height = image_r2.shape
        NI = np.zeros((width - 2 * r2, height - 2 * r2))
        RD = np.zeros((width - 2 * r2, height - 2 * r2))
        r1 = r2 - 1

        # k = 0
        for i in range(r2,height - r2):
            for j in range(r2,width -r2):
                area = image_r2[i - r2  : i  + r2 + 1 , j  - r2 :j + r2 + 1]
                sum_r2_patch = np.sum(area)
                # k += 1
                r1_descriptor = self.getInterNeighbors(image_r1, r1, i, j)
                r2_descriptor = self.getInterNeighbors(image_r2, r2, i, j)

                # r1_descriptor_str = str(r1_descriptor)
                # with open("r1_inter", "a") as file:
                #     file.write(r1_descriptor_str + "\n")

                # r2_descriptor_str = str(r2_descriptor)
                # with open("r2_inter", "a") as file:
                #     file.write(r2_descriptor_str + "\n")
                #     file.write("\n")
                # for i in range(0, 9):
                #     print(r2_descriptor)
                # return
                NI[i - r2, j - r2] = self.getNIDescriptor(r2_descriptor, r2, sum_r2_patch)
                RD[i - r2, j - r2] = self.getRDDescriptor(r2_descriptor, r1_descriptor)

        return NI, RD


lbp = MRELBP()


In [4]:
def chiSquared(p,q):
    return np.sum((p-q)**2/(p+q+1e-6))

In [5]:
def make_train(root_path):
    decision_path = os.path.join(root_path, "000")
    image_path = os.path.join(root_path, "images")
    with open(decision_path + "/classes.txt", "r") as file:
        numClasses = int(file.readline())
        classes = {}
        for line in file:
            columns = line.split()
            classes[int(columns[1])] = columns[0]
    
    print("Training steps:")
    with open(decision_path + "/test.txt", "r") as train:
        numTrain = int(train.readline())
        mrelbp_hist_800 = []
        labels = []
        i = 0
        for current_line_number, line in enumerate(train):
            i = i + 1        
            labels.append(line.split()[1])
            file_name = line.split()[0]

            path_file = f'{image_path}/{file_name}'
            print(path_file)
            img = cv2.imread(path_file.rstrip(), cv2.IMREAD_GRAYSCALE)
            img_gray = cv2.resize(img, (img_width, img_height), interpolation = cv2.INTER_LANCZOS4)
            combined_hist_800 = lbp.MRELBP(img_gray)
            mrelbp_hist_800.append(combined_hist_800)
            print(i)
            if i >= numTrain:
                np.save(f'{root_path}/X_800_batch.npy', np.array(mrelbp_hist_800))
                np.save(f'{root_path}/label_batch.npy', np.array(labels))
                break

    SVM_classifier_mrelbp = make_pipeline(StandardScaler(), SVC(gamma='auto'))
    SVM_classifier_mrelbp.fit(mrelbp_hist_800,labels)
    result_labels_and_hist_800 = []
    print("testing steps:")
    with open(decision_path + "/train.txt", "r") as train:
        print(train.readline())
        # numTrain = int(train.readline())
        test_hist_800 = []
        test_labels = []
        i = 0  
        for current_line_number, line in enumerate(train):
            i = i + 1        
            test_labels.append(line.split()[1])
            file_name = line.split()[0]

            path_file = f'{image_path}/{file_name}'
            print(path_file)
            img = cv2.imread(path_file.rstrip(), cv2.IMREAD_GRAYSCALE)
                    # print(img)
            img_gray = cv2.resize(img, (img_width, img_height), interpolation = cv2.INTER_LANCZOS4)
            combined_hist_800 = lbp.MRELBP(img_gray)
            test_hist_800.append(combined_hist_800)
            result_labels_and_hist_800.append((line.split()[1], combined_hist_800))
            print(i)
            if i >= 480:
                np.save(f'{root_path}/test_800_batch.npy', np.array(result_labels_and_hist_800, dtype=object))
                break

    result_SVM_labels_mrelbp_hist = []
    
    for m in range(480):
        result_SVM_labels_mrelbp_hist.append((result_labels_and_hist_800[m][0],
                    SVM_classifier_mrelbp.predict([result_labels_and_hist_800[m][1].tolist()])))
    pred = 0
    for j in range(480):
        pred += (int(result_SVM_labels_mrelbp_hist[j][0] == result_SVM_labels_mrelbp_hist[j][1][0]))
        
    print('predictions =',pred)
    print('accuracy svm=',pred/(480)*100)

    NB_classifier_mrelbp = MultinomialNB()
    LR_classifier_mrelbp = LogisticRegression(random_state=0)
    KNN_classifier_mrelbp = KNeighborsClassifier(n_neighbors=3, weights='uniform', algorithm='auto', metric=chiSquared)

    result_NB_labels_mrelbp_hist = []
    result_LR_labels_mrelbp_hist = []
    result_KNN_labels_mrelbp_hist = []

    NB_classifier_mrelbp.fit(mrelbp_hist_800,labels)
    LR_classifier_mrelbp.fit(mrelbp_hist_800,labels)
    KNN_classifier_mrelbp.fit(mrelbp_hist_800,labels)

    for m in range(480):
        result_NB_labels_mrelbp_hist.append((result_labels_and_hist_800[m][0],
                        NB_classifier_mrelbp.predict([result_labels_and_hist_800[m][1].tolist()])))
        result_LR_labels_mrelbp_hist.append((result_labels_and_hist_800[m][0],
                        LR_classifier_mrelbp.predict([result_labels_and_hist_800[m][1].tolist()])))
        result_KNN_labels_mrelbp_hist.append((result_labels_and_hist_800[m][0],
                    KNN_classifier_mrelbp.predict([result_labels_and_hist_800[m][1].tolist()])))
        
    pred = 0
    for j in range(480):
        pred += (int(result_NB_labels_mrelbp_hist[j][0] == result_NB_labels_mrelbp_hist[j][1][0]))
        
    print('predictions =',pred)
    print('accuracy nb =',pred/(480)*100)



    pred = 0
    for j in range(480):
        pred += (int(result_LR_labels_mrelbp_hist[j][0] == result_LR_labels_mrelbp_hist[j][1][0]))
        
    print('predictions =',pred)
    print('accuracy lr =',pred/(480)*100)

    pred = 0
    for j in range(480):
        pred += (int(result_KNN_labels_mrelbp_hist[j][0] == result_KNN_labels_mrelbp_hist[j][1][0]))
        
    print('predictions =',pred)
    print('accuracy knn =',pred/(480)*100)

In [6]:
Dataset_dir ='D:\\Thesis\\CodeTest\\python\\Train\\Dataset'



In [7]:
Outex_11n = os.path.join(Dataset_dir, "Outex-TC-00011n")
Outex_11n

'D:\\Thesis\\CodeTest\\python\\Train\\Dataset\\Outex-TC-00011n'

In [8]:
make_train(Outex_11n)

Training steps:
D:\Thesis\CodeTest\python\Train\Dataset\Outex-TC-00011n\images/000020.bmp
Execution Time: 6.0314 seconds
1
D:\Thesis\CodeTest\python\Train\Dataset\Outex-TC-00011n\images/000021.bmp
Execution Time: 5.0761 seconds
2
D:\Thesis\CodeTest\python\Train\Dataset\Outex-TC-00011n\images/000022.bmp
Execution Time: 4.9626 seconds
3
D:\Thesis\CodeTest\python\Train\Dataset\Outex-TC-00011n\images/000023.bmp
Execution Time: 4.9122 seconds
4
D:\Thesis\CodeTest\python\Train\Dataset\Outex-TC-00011n\images/000024.bmp
Execution Time: 4.7990 seconds
5
D:\Thesis\CodeTest\python\Train\Dataset\Outex-TC-00011n\images/000025.bmp
Execution Time: 4.9931 seconds
6
D:\Thesis\CodeTest\python\Train\Dataset\Outex-TC-00011n\images/000026.bmp
Execution Time: 4.7986 seconds
7
D:\Thesis\CodeTest\python\Train\Dataset\Outex-TC-00011n\images/000027.bmp
Execution Time: 4.8453 seconds
8
D:\Thesis\CodeTest\python\Train\Dataset\Outex-TC-00011n\images/000028.bmp
Execution Time: 5.1208 seconds
9
D:\Thesis\CodeTest\py

c:\Users\gbmhi\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


predictions = 356
accuracy nb = 74.16666666666667
predictions = 321
accuracy lr = 66.875
predictions = 382
accuracy knn = 79.58333333333333


In [9]:
Outex_11r= os.path.join(Dataset_dir, "Outex-TC-00011r")
Outex_11r

'D:\\Thesis\\CodeTest\\python\\Train\\Dataset\\Outex-TC-00011r'

In [10]:
make_train(Outex_11r)

Training steps:
D:\Thesis\CodeTest\python\Train\Dataset\Outex-TC-00011r\images/000020.bmp
Execution Time: 7.2220 seconds
1
D:\Thesis\CodeTest\python\Train\Dataset\Outex-TC-00011r\images/000021.bmp
Execution Time: 10.0408 seconds
2
D:\Thesis\CodeTest\python\Train\Dataset\Outex-TC-00011r\images/000022.bmp


KeyboardInterrupt: 